Paper: https://www.andrew.cmu.edu/user/lakoglu/pubs/StackOverflow-churn.pdf

Description of datasets: https://ia800107.us.archive.org/27/items/stackexchange/readme.txt

Site for download of datasets: https://archive.org/details/stackexchange

This code has 6 steps

    1. Load StackOverflow datasets as dataframe
    2. Extract and label the datasets for each task
    3. Extract features for each task
    4. Analyze features|
    5. Train models for each task with the features
    6. Quantify the importance of each feature category

1. Load StackOverflow datasets as dataframe

In [1]:
import sys
!{sys.executable} -m pip --proxy http://10.241.3.7:8080 install xmltodict

In [2]:
import xmltodict
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
xml_dir = 'stackoverflow.com/'  ### Full dataset
xml_dir = 'math.stackexchange.com/'   ### Small dataset
xml_dir = 'academia.meta.stackexchange.com/'   ### Tiny dataset

In [427]:
# Read xml file and transform to pandas dataframe

import json

def xml2df(xml_path):
    f = open(xml_path, 'r', encoding='UTF8')
    data = f.read()
    f.close()
    xml_dict = xmltodict.parse(data)
    key = list(xml_dict.keys())[0]
    df = pd.DataFrame(xml_dict[key]['row'])
    df.columns = [col.replace('@', '') for col in df.columns]
    return df

In [467]:
# 1. Read Users.xml
xml_path = xml_dir + 'Users.xml'
users_df = xml2df(xml_path)

# 2. Change data type of columns
users_df = users_df[['Id', 'Reputation', 'CreationDate', 'LastAccessDate']]
users_df.CreationDate = pd.to_datetime(users_df.CreationDate)
users_df.LastAccessDate = pd.to_datetime(users_df.LastAccessDate)
users_df.Id = users_df['Id'].astype('int64')
users_df.Reputation = users_df.Reputation.astype('int64')
users_df = users_df.set_index('Id')
users_df = users_df.drop([-1])

print(users_df)

KeyError: '[-1] not found in axis'

In [468]:
# 1. Read Posts.xml
xml_path = xml_dir + 'Posts.xml'
posts_df = xml2df(xml_path)

# 2. Change data type of columns
posts_df = posts_df[['Id', 'CreationDate', 'AcceptedAnswerId', 'Score', 'OwnerUserId']]
posts_df.CreationDate = pd.to_datetime(posts_df.CreationDate)
posts_df.Id = posts_df.Id.astype('int64')
posts_df = posts_df.dropna(subset=['OwnerUserId'])
posts_df.OwnerUserId = posts_df.OwnerUserId.astype('int64')
posts_df = posts_df.set_index('Id')
print(posts_df)

                CreationDate AcceptedAnswerId Score  OwnerUserId
Id                                                              
1    2012-02-14 20:39:10.140              NaN     4            5
2    2012-02-14 20:41:04.273              NaN     3            5
4    2012-02-14 21:37:15.053                5     3           30
5    2012-02-14 21:46:33.213              NaN     5           23
6    2012-02-14 22:07:56.987              NaN     4           31
...                      ...              ...   ...          ...
4526 2019-08-21 05:58:07.673              NaN    15          118
4527 2019-08-21 06:07:55.123              NaN     7        13240
4528 2019-08-26 01:57:43.213              NaN     3        21968
4529 2019-08-26 06:06:01.747              NaN     1         7734
4530 2019-08-27 16:06:35.500              NaN     4           73

[3068 rows x 4 columns]


In [76]:
# Save and Load dataframe

def save_df(df, filename):
    df.to_pickle("{}.pkl".format(filename))

    
def load_df(filename):
    return pd.read_pickle("{}.pkl".format(filename))


2. Extract and label the datasets for each task

You should extract the dataset for the period of the dataset: July 31, 2008 ~  July 31, 2012 

There are 2 tasks:

    A. After a user's K-th post, predict how likely it is that the user will churn
    B. After the T-th day from the account creation of a user, predict how likely it is that the user will churn

In [456]:
# You should extract the dataset for the period of the dataset: July 31, 2008 ~  July 31, 2012

start_time = pd.to_datetime('2008-07-31')
end_time = pd.to_datetime('2012-07-31')

posts_df = posts_df[(posts_df['CreationDate'] >= start_time) & (posts_df['CreationDate'] <= end_time)]
users_df = users_df[(users_df['CreationDate'] >= start_time) & (users_df['CreationDate'] <= end_time)]

print(posts_df)
print(users_df)

               CreationDate AcceptedAnswerId Score  OwnerUserId
Id                                                             
1   2012-02-14 20:39:10.140              NaN     4            5
2   2012-02-14 20:41:04.273              NaN     3            5
4   2012-02-14 21:37:15.053                5     3           30
5   2012-02-14 21:46:33.213              NaN     5           23
6   2012-02-14 22:07:56.987              NaN     4           31
..                      ...              ...   ...          ...
155 2012-07-08 17:35:27.660              NaN     1           65
156 2012-07-14 08:48:00.980              NaN     5           81
157 2012-07-23 07:33:01.473              160    10          929
159 2012-07-23 16:56:29.283              NaN     5         1033
160 2012-07-23 21:43:41.267              NaN    11          346

[132 rows x 4 columns]
      Reputation        CreationDate          LastAccessDate
Id                                                          
2            101 2012-

In [457]:
# Dataset in Task 1
#   Posts: Extract K posts of each user
#   Users: Extract users who post at least K

def getTask1Posts(posts, K=20):
    group_posts = posts.groupby('OwnerUserId')
    kth_group_posts = group_posts.nth(K)
    return kth_group_posts.reset_index()

def getTask1Users(users, posts, K=20):
    kth_posts = getTask1Posts(posts, K)
    return users[users.index.isin(kth_posts.OwnerUserId)]

ex1 = getTask1Posts(posts_df, K=5)
print(ex1)

ex2 = getTask1Users(users_df, posts_df, K=5)
print(ex2)

   OwnerUserId            CreationDate AcceptedAnswerId Score
0            5 2012-02-16 01:25:12.627              NaN     3
1           23 2012-05-24 20:47:38.473              NaN     1
2           53 2012-03-07 21:38:42.657              NaN     7
3           73 2012-02-21 20:04:30.667               38     4
4           96 2012-03-07 05:03:26.243              NaN     2
    Reputation        CreationDate          LastAccessDate
Id                                                        
5         1969 2012-02-14 20:22:08 2014-05-24 05:16:16.957
23         101 2012-02-14 20:33:51 2019-01-31 21:34:44.323
53      162384 2012-02-14 21:47:58 2019-02-08 21:12:16.727
73       42744 2012-02-15 00:04:47 2019-08-28 17:51:31.323
96       24181 2012-02-15 11:27:39 2018-08-27 15:03:34.713


In [461]:
# Dataset in Task 2
#   Users: Extract users who post at least 1
#   Posts: Extract posts which create before T day from the account creation of the owner

def getTask2Posts(users, posts, T=30):
    users = getTask1Users(users, posts, 1)
    posts = posts[posts.OwnerUserId.isin(users.index)]
    backup_index = posts.index

    posts = posts.reset_index(drop=True)
    posts['UserCreationDate'] = users.loc[posts.OwnerUserId, 'CreationDate'].reset_index(drop=True)
    posts['DiffCreationDate'] = (posts.CreationDate - posts.UserCreationDate).dt.days
    posts.index = backup_index
    print(posts)
    
    posts = posts[posts.DiffCreationDate <= T]
    print(posts)
    #posts.drop(columns=['UserCreationDate', 'DiffCreationDate'])

    return posts

ex3 = getTask2Posts(users_df, posts_df, T=30)
print(ex3)

               CreationDate AcceptedAnswerId Score  OwnerUserId  \
Id                                                                
1   2012-02-14 20:39:10.140              NaN     4            5   
2   2012-02-14 20:41:04.273              NaN     3            5   
5   2012-02-14 21:46:33.213              NaN     5           23   
6   2012-02-14 22:07:56.987              NaN     4           31   
7   2012-02-14 22:11:59.587               26     9           31   
..                      ...              ...   ...          ...   
151 2012-06-27 19:01:47.187              NaN     4           53   
152 2012-07-04 05:07:45.260              154     5          411   
153 2012-07-04 12:49:41.820              NaN     6          929   
157 2012-07-23 07:33:01.473              160    10          929   
160 2012-07-23 21:43:41.267              NaN    11          346   

       UserCreationDate  DiffCreationDate  
Id                                         
1   2012-02-14 20:22:08                 

In [304]:
# Churn in Task 1
#   Churners: Users who did not post for at least 6 months from their K-th post 
#   Stayers:  Users who created at least one post within the 6 months from their K-th post

def getTask1Labels(users, posts, K=20):
    label_df = getTask1Users(users, posts, K=K).set_index('@Id')
    label_df = label_df.drop(label_df.columns, axis=1)
    
    posts_k = getTask1Posts(posts, K=K).set_index('@OwnerUserId')
    posts_k_1 = getTask1Posts(posts, K=K + 1).set_index('@OwnerUserId')
    diff = (posts_k_1.loc[posts_k.index, '@CreationDate'] - posts_k.loc[:, '@CreationDate']).fillna(pd.Timedelta(days=181))

    label_df['is_churn'] = 0.0
    label_df[diff.loc[label_df.index] > pd.Timedelta(days=180)] = 1.0
    return label_df

ex4 = getTask1Labels(users_df, posts_df, 3)
print(ex4)

     is_churn
@Id          
5         0.0
23        0.0
53        0.0
66        1.0
73        0.0
96        0.0
118       0.0
346       0.0


In [365]:
# Churn in Task2
#   Churners: Users who did not post for at least 6 months from T days after account creation
#   Stayers:  Users who created at least one post within the 6 months from T days after account creation

def getTask2Labels(users, posts, T=30):
    label_df = getTask1Users(users, posts, K=1).set_index('@Id')
    label_df = label_df.drop(label_df.columns, axis=1)
    
    posts_t = getTask2Posts(users, posts, T=T)
    posts_t_size = posts_t.groupby('@OwnerUserId').size()
    label_df = posts_t_size.index
    posts_t_180 = getTask2Posts(users, posts, T=T + 180)
    posts_t_180_size = posts_t_180.groupby('@OwnerUserId').size()
    new_posts_size = posts_t_180_size.loc[label_df] - posts_t_size[label_df]
    
    label_df = pd.DataFrame(index=label_df)
    label_df['is_churn'] = 1.0
    label_df[new_posts_size > 0] = 0.0

    return label_df

ex5 = getTask2Labels(users_df, posts_df, T=30)
print(ex5)

              is_churn
@OwnerUserId          
100                1.0
118                1.0
201                1.0
23                 0.0
3                  1.0
31                 1.0
324                1.0
346                0.0
49                 0.0
5                  1.0
53                 0.0
62                 0.0
66                 1.0
73                 0.0
8                  1.0
929                0.0
96                 0.0


3. Extract features for each task

3-1. Temporal features

In [380]:
# Temporal features 1: gap1
def getTimeGap1OfUser(users, posts):
    users = getTask1Users(users, posts, K=1).set_index('@Id')
    posts = getTask1Posts(posts, K=1).set_index('@OwnerUserId')
    gap1 = posts.loc[users.index, '@CreationDate'] - users['@CreationDate']
    return gap1

ex6 = getTimeGap1OfUser(users_df, posts_df)
print(ex6)

@Id
3       6 days 02:42:51.487000
5       0 days 00:18:56.273000
8       1 days 17:32:21.850000
23     12 days 03:52:03.527000
31      0 days 01:25:14.587000
49     10 days 03:11:47.340000
53      6 days 22:36:11.110000
62     97 days 13:16:38.020000
66     21 days 02:29:28.463000
73      0 days 15:07:53.093000
96      0 days 23:17:03.140000
100     0 days 03:10:27.780000
118     5 days 18:16:16.567000
201     4 days 22:56:53.570000
319   114 days 03:05:11.720000
324     4 days 00:02:55.620000
346    65 days 13:00:00.083000
411   111 days 20:40:23.260000
929    39 days 22:57:25.820000
Name: @CreationDate, dtype: timedelta64[ns]


In [382]:
# Temporal features 2: gapK
def getTimeGapsOfPosts(posts, K):
    posts_k1 = getTask1Posts(posts, K=K-1).set_index('@OwnerUserId')
    posts_k = getTask1Posts(posts, K=K).set_index('@OwnerUserId')
    
    gapk = posts_k['@CreationDate'] - posts_k1.loc[posts_k.index, '@CreationDate']
    return gapk

ex7 = getTimeGapsOfPosts(posts_df, K=5)
print(ex7)

@OwnerUserId
23    0 days 00:00:09.893000
5     0 days 00:11:36.894000
53   13 days 05:41:52.664000
73    5 days 17:47:39.614000
96    9 days 07:21:47.410000
Name: @CreationDate, dtype: timedelta64[ns]


In [ ]:
# Temporal features 3: last_gap
def getTimeLastGapOfPosts(posts):
    return


In [ ]:
# Temporal features 4: time_since_last_post
def getTimeSinceLastPost(users, posts):
    return


In [ ]:
# Temporal features 5: mean_gap
def getTimeMeanGap(posts):
    return


3-2. Frequency features

In [ ]:
# Frequency features 1: num_answers
# Frequency features 2: num_questions
def getNumAnswers(posts):
    return

def getNumQuestions(posts):
    return


In [ ]:
# Frequency features 3: ans_ques_ratio
def getAnsQuesRatio(num_answers, num_questions):
    return


In [ ]:
# Frequency features 4: num_posts
def getNumPosts(posts):
    return


3-3. Knowledge features

In [ ]:
# Knowledge features 1: accepted_answerer_rep
def getRepOfAcceptedAnswerer(users, posts):
    return


In [ ]:
# Knowledge features 2: max_rep_answerer 
def getMaxRepAmongAnswerer(users, posts):
    return


In [ ]:
# Knowledge features 3: num_que_answered
def getNumQueAnswered(posts):
    return


In [ ]:
# Knowledge features 4: time_for_first_ans
def getTimeForFirstAns(posts):
    return


In [ ]:
# Knowledge features 5: rep_questioner
def getAvgRepOfQuestioner(users, posts):
    return


In [ ]:
# Knowledge features 6: rep_answerers
def getAvgRepOfAnswerer(users, posts):
    return


In [ ]:
# Knowledge features 7: rep_co_answerers
def getAvgRepOfCoAnswerer(users, posts):
    return


In [ ]:
# Knowledge features 8: num_answers_recvd
def getAvgNumAnsReceived(posts):
    return


3-4. Speed features

In [ ]:
# Speed features 1: answering_speed
def getAnsweringSpeed(posts):
    return


3-5. Quality features

In [ ]:
# Quality features 1: ans_score
# Quality features 2: que_score
def getScoreOfAnswers(posts):
    return

def getScoreOfQuestions(posts):
    return


3-6. Consistency features

In [ ]:
# Consistency features 1: ans_stddev
# Consistency features 2: que_stddev
def getStdevOfScoresOfAnswers(posts):
    return

def getStdevOfScoresOfQuestions(posts):
    return


3-7. Gratitude features

In [ ]:
# Gratitude features 1: ans_comments
# Gratitude features 2: que_comments
def getAvgNumOfAnswers(posts):
    return

def getAvgNumOfQuestions(posts):
    return


3-8. Competitiveness features

In [ ]:
# Competitiveness features 1: relative_rank_pos
def getRelRankPos(posts):
    return


3-9. Content features

In [ ]:
# Content features 1: ans_length
# Content features 2: que_length
def getLengthOfAnswers(posts):
    return

def getLengthOfQuestions(posts):
    return


3-10. Extract all features for each task

In [ ]:
def getFeatures(features, users, posts, task, K=None, T=None):
    assert(task in [1,2])
    
    if -1 in features.index:
        features = features.drop([-1])
    
    return features

In [ ]:
task1_features = []
for K in range(1, 20+1):
    task1_features.append()
    
task2_features = []
for T in [7, 15, 30]:
    task2_features.append()

4. Analyze features


In [ ]:
# Figure 2: Gap between posts
#    For a user who churns, gap between consecutive posts keeps increasing. 
#    Gaps for those who stay are much lower, and stabilize around 20,000 minutes,
#      indicating routine posting activity in every ≈2 weeks.

for K in range(2, 21):
    pass

In [ ]:
# Figure 3: # Answers vs Churn probability
#    The probability of churning for a user decreases the more answers s/he provides.
#    It is even lower if s/he asks more questions alongside.

for features in task2_features:
    pass

In [ ]:
# Figure 4: K vs Time taken for the first answer to arrive
#    The more the time taken for a user to receive an answer, 
#      the lesser the satisfaction level and the more the chances of churning.


5. Train models for each task with the features

    1. Decision Tree
    2. SVM (Linear)
    3. SVM (RBF)
    4. Logistic Regression
    

In [ ]:
# Table 2: Performance on Task 1

from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold

seed = 1234

for i, features in enumerate(task1_features):
    pass

In [ ]:
# Table 3: Performance on Task 2

for i, features in enumerate(task2_features):
    pass

6. Quantify the importance of each feature category


In [ ]:
# Table 4: Temporal Features Analysis

for i, features in enumerate(task1_features):
    pass

In [ ]:
# Figure 5: Churn prediction accuracy when features from each category are used in isolation
